In [33]:
from vgg19 import VGG19
from keras.preprocessing import image
from imagenet_utils import preprocess_input
from keras.models import Model, load_model
import numpy as np
import cv2

In [34]:
base_model = load_model('vgg19.h5')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [35]:
model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  """Entry point for launching an IPython kernel.


In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [45]:
def repool(path, grayornot=False, normalornot=False, num=1):
    img = image.load_img(path, grayscale=grayornot)
    
    d_img = np.array(img)
    step_i,step_j = d_img.shape[:2]
    step_i /= num
    step_j /= num
    fmap = np.zeros((1, 100352))
    for i in range(num):
        for j in range(num):
            g = d_img[int(0+i*step_i):int(step_i+i*step_i), int(0+j*step_j):int(step_j+j*step_j)]
            g = cv2.resize(g, (224,224))

            #x = image.img_to_array(img)
            #x = np.expand_dims(x, axis=0)
            if grayornot == True:
                img2 = np.zeros_like(g)
                img2 = np.stack([g]*3, axis=2)
                img2[:,:,0] = g[:,:]
                img2[:,:,1] = g[:,:]
                img2[:,:,2] = g[:,:]
                g = img2
            #x = preprocess_input(x)
            if normalornot == True:
                g = [[g/.255]]
            else:
                g = [[g]]
            block4_pool_features = model.predict(g).flatten()
            fmap += block4_pool_features

    return fmap

In [42]:
fea_n = []
for file in folder:
    fea = repool('../../../PosData/'+file, grayornot=False, normalornot=False, num=1).flatten()
    fea_n.append([file, fea])

fea_n = np.array(fea_n, dtype=object)
np.save('data_color_1x1',np.asarray(fea_n))

In [46]:
from os import listdir

folder = listdir('../../../Database/')
result = []

for subf in folder: 
    try:
        subfolder = listdir('../../../Database/'+subf)
        for tgimg in subfolder:
            try:
                fea = repool('../../../Database/'+subf+'/'+tgimg, grayornot=False, normalornot=False, num=1).flatten()
                
                result.append([subf+":"+tgimg,fea])
            except:
                a=0
    except:
        a=0

fea_n = np.array(result, dtype=object)
np.save('data_real_color',np.asarray(fea_n))

In [49]:
np.load('data_real_color.npy')[0]

array(['G-M:IMG_5508.JPG',
       array([0.        , 0.        , 4.04795122, ..., 0.        , 0.        ,
       0.        ])], dtype=object)